# Ensemble Models

- Mainly we have ensemble the predicted results of two type Models
    - XGBOOST
    - Convolution Neural Network (CNN)
- XGBOOST combined R2 is 0.47
- CNN combined R2 is 0.54

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/MyDrive/enz-eff-project
!pip install -r requirements.txt

/content/drive/.shortcut-targets-by-id/1iS6gSWfUE3cZnmrNWbbV9_W_zQH3vaiu/enz-eff-project


In [ ]:
%cd improved_code/Inference

/content/drive/.shortcut-targets-by-id/1iS6gSWfUE3cZnmrNWbbV9_W_zQH3vaiu/enz-eff-project/improved_code/Inference


In [ ]:
import numpy as np
import tensorflow as tf
import random

# Set seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)
random.seed(42)

import pickle
import pandas as pd
import numpy as np
import warnings
import xgboost as xgb
from sklearn.metrics import mean_squared_error, r2_score
from scipy import stats
from os.path import join
from sklearn import preprocessing
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from tensorflow.keras.models import load_model
from scipy.optimize import minimize


warnings.filterwarnings("ignore")

# loading dataset
data_train = pd.read_pickle(
    "../../data/kcat_data/splits/train_df_kcat_new.pkl"
)


data_test = pd.read_pickle(
    "../../data/kcat_data/splits/test_df_kcat_new.pkl"
)

len(data_train), len(data_test)

(3391, 874)

In [ ]:
data_train.rename(columns={"geomean_kcat": "log10_kcat"}, inplace=True)
data_test.rename(columns={"geomean_kcat": "log10_kcat"}, inplace=True)
test_Y = np.array(list(data_test["log10_kcat"]))

## XBOOST models

### Ensemble with our features

> Ensemble model output:
{'Mean Squared Error': 0.7008645270806269, 'R-squared': 0.4757678843460912, 'Pearson coefficient': 0.6974741404297905}

In [ ]:
def load_pickle_model(path):
    with open(path, "rb") as file:
        model = pickle.load(file)
    return model


def get_model_path(model_name):
    xgboost_models_path = "../../models/best_models/xgboost"
    model_path = join(xgboost_models_path, model_name)
    return model_path


def get_esm1b_xgboost_pred(model_path):
    test_X = np.array(list(data_test["ESM1b"]))
    dtest = xgb.DMatrix(test_X)
    model = load_pickle_model(model_path)
    y_pred = model.predict(dtest)

    return y_pred


def get_esm1b_ts_xgboost_pred(model_path):
    test_X = np.array(list(data_test["ESM1b_ts"]))
    dtest = xgb.DMatrix(test_X)
    model = load_pickle_model(model_path)
    y_pred = model.predict(dtest)
    return y_pred


def get_drfp_xgboost_pred(model_path):
    test_X = np.array(list(data_test["DRFP"]))
    dtest = xgb.DMatrix(test_X)
    model = load_pickle_model(model_path)
    y_pred = model.predict(dtest)
    return y_pred


def get_diff_xgboost_pred(model_path):
    test_X = np.array(list(data_test["difference_fp"]))
    dtest = xgb.DMatrix(test_X)
    model = load_pickle_model(model_path)
    y_pred = model.predict(dtest)
    return y_pred


def get_structural_fp_xgboost_pred(model_path):
    test_X = np.array(list(data_test["structural_fp"]))
    dtest = xgb.DMatrix(test_X)
    model = load_pickle_model(model_path)
    y_pred = model.predict(dtest)
    return y_pred


def get_esm1b_ts_drfp_xgboost_pred(model_path):
    drfp = np.array(list(data_test["DRFP"]))
    esm1b_ts = np.array(list(data_test["ESM1b_ts"]))
    test_X = np.concatenate([drfp, esm1b_ts], axis=1)
    model = load_pickle_model(model_path)
    y_pred = model.predict(xgb.DMatrix(test_X))
    return y_pred


def get_esm1b_ts_diff_fp_xgboost_pred(model_path):
    drfp = np.array(list(data_test["difference_fp"]))
    esm1b_ts = np.array(list(data_test["ESM1b_ts"]))
    test_X = np.concatenate([drfp, esm1b_ts], axis=1)
    model = load_pickle_model(model_path)
    y_pred = model.predict(xgb.DMatrix(test_X))
    return y_pred

def get_all_features_xgboost_pred(model_path):
    data_train = pd.read_pickle(join("..", "..", "data", "kcat_data", "splits", "train_df_kcat_ours.pkl"))
    data_test = pd.read_pickle(join("..", "..", "data", "kcat_data", "splits", "test_df_kcat_ours.pkl"))
    data_train.rename(columns = {"geomean_kcat" :"log10_kcat"}, inplace = True)
    data_test.rename(columns = {"geomean_kcat" :"log10_kcat"}, inplace = True)
    train_X = np.array(list(data_train["ESM1b"]))
    test_X = np.array(list(data_test["ESM1b"]))
    for col in data_train.columns[17:21]:
      if col == 'structural_fp':
        train_X = np.concatenate([train_X, np.array(data_train[col].tolist(), dtype='float64')], axis = 1)
        test_X = np.concatenate([test_X, np.array(data_test[col].tolist(), dtype='float64')], axis = 1)
      else:
        train_X = np.concatenate([train_X, np.array(list(data_train[col]), dtype='float64')], axis = 1)
        test_X = np.concatenate([test_X, np.array(list(data_test[col]), dtype='float64')], axis = 1)
    train_X = np.concatenate([train_X, np.array(list(data_train['ESM1b_norm']), dtype='float64')], axis = 1)
    test_X = np.concatenate([test_X, np.array(list(data_test['ESM1b_norm']), dtype='float64')], axis = 1)
    train_X = np.concatenate([train_X, np.array(list(data_train['ESM1b_ts_norm']), dtype='float64')], axis = 1)
    test_X = np.concatenate([test_X, np.array(list(data_test['ESM1b_ts_norm']), dtype='float64')], axis = 1)
    train_X = np.concatenate([train_X, np.array(data_train.iloc[:,22:-7], dtype='float64')], axis = 1)
    test_X = np.concatenate([test_X, np.array(data_test.iloc[:,22:-7], dtype='float64')], axis = 1)
    train_X = train_X.astype('float64')
    test_X = test_X.astype('float64')
    train_Y = np.array(list(data_train["log10_kcat"]))
    test_Y = np.array(list(data_test["log10_kcat"]))
    model = load_pickle_model(model_path)
    y_pred = model.predict(xgb.DMatrix(test_X))
    return y_pred

def evaluate_models_weight(weights, model_preds, true_values):
    weighted_avg = np.average(model_preds, weights=weights, axis=0)
    r2 = r2_score(true_values, weighted_avg)
    return -r2  # We want to maximize R2, so minimize -R2


def calculate_weighted_mean(model_preds, true_values):
    num_models = model_preds.shape[0]
    # Initial weights (for illustration, you can adjust this as needed)
    initial_weights = np.ones(num_models) / num_models
    print(initial_weights)
    print(initial_weights.shape)
    # Constraint: the sum of weights must be 1
    constraints = {"type": "eq", "fun": lambda w: np.sum(w) - 1.0}
    # Bounds for weights (between 0 and 1)
    bounds = [(0, 1)] * num_models
    print("bounds shape:", bounds)
    result = minimize(
        evaluate_models_weight,
        initial_weights,
        args=(model_preds, true_values),
        method="SLSQP",
        bounds=bounds,
        constraints=constraints,
    )
    best_weights = result.x
    weighted_avg_pred = np.average(model_preds, weights=best_weights, axis=0)
    return weighted_avg_pred


def get_final_results(y_test, y_pred):
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    Pearson, _ = stats.pearsonr(np.reshape(y_test, -1), y_pred)
    output = {
        "Mean Squared Error": mse,
        "R-squared": r2,
        "Pearson coefficient": Pearson,
    }
    print(f"Ensemble model output:\n{output}")
    return output


def ensemble_xgboost_models():
    esm1b_pred = get_esm1b_xgboost_pred(
        get_model_path("xgboost_esm1b_r2_39_mse_81.h5"))

    esm1b_ts_pred = get_esm1b_ts_xgboost_pred(
        get_model_path("xgboost_esm1b_ts_r2_41_mse_77.h5")
    )
    diff_fp_pred = get_diff_xgboost_pred(
        get_model_path("xgboost_diff_r2_35_mse_86.h5")
        )

    structural_fp_pred = get_structural_fp_xgboost_pred(
        get_model_path("xgboost_structFp_r2_31_mse_91.h5")
    )

    drfp_fp_pred = get_drfp_xgboost_pred(
        get_model_path("xgboost_drfp_r2_35_mse_86.h5")
    )

    esm1b_ts_drfp_pred = get_esm1b_ts_drfp_xgboost_pred(
        get_model_path("xgboost_esm1bts_drfp_r2_43_mse_76_pc_65.h5")
    )

    esm1b_ts_diff_pred = get_esm1b_ts_diff_fp_xgboost_pred(
        get_model_path("xgboost_esm1b_ts_diff_r2_41_mse_77.h5")
    )

    all_pred = get_all_features_xgboost_pred(
        get_model_path("xgboost_all.h5")
    )
    models_pred = np.array(
        [
            esm1b_pred,
            esm1b_ts_pred,
            diff_fp_pred,
            structural_fp_pred,
            drfp_fp_pred,
            esm1b_ts_drfp_pred,
            esm1b_ts_diff_pred,
            all_pred
        ]
    )

    weighted_avg_pred = calculate_weighted_mean(models_pred, test_Y)
    get_final_results(test_Y, weighted_avg_pred)


if __name__ == "__main__":
    ensemble_xgboost_models()


[0.125 0.125 0.125 0.125 0.125 0.125 0.125 0.125]
(8,)
bounds shape: [(0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1)]
Ensemble model output:
{'Mean Squared Error': 0.7008645270806269, 'R-squared': 0.4757678843460912, 'Pearson coefficient': 0.6974741404297905}


### Ensemble without our features

> Ensemble model output:
{'Mean Squared Error': 0.700865090031929, 'R-squared': 0.4757674632702046, 'Pearson coefficient': 0.6974826296893911}

In [ ]:
def load_pickle_model(path):
    with open(path, "rb") as file:
        model = pickle.load(file)
    return model


def get_model_path(model_name):
    xgboost_models_path = "../../models/best_models/xgboost"
    model_path = join(xgboost_models_path, model_name)
    return model_path


def get_esm1b_xgboost_pred(model_path):
    test_X = np.array(list(data_test["ESM1b"]))
    dtest = xgb.DMatrix(test_X)
    model = load_pickle_model(model_path)
    y_pred = model.predict(dtest)

    return y_pred


def get_esm1b_ts_xgboost_pred(model_path):
    test_X = np.array(list(data_test["ESM1b_ts"]))
    dtest = xgb.DMatrix(test_X)
    model = load_pickle_model(model_path)
    y_pred = model.predict(dtest)
    return y_pred


def get_drfp_xgboost_pred(model_path):
    test_X = np.array(list(data_test["DRFP"]))
    dtest = xgb.DMatrix(test_X)
    model = load_pickle_model(model_path)
    y_pred = model.predict(dtest)
    return y_pred


def get_diff_xgboost_pred(model_path):
    test_X = np.array(list(data_test["difference_fp"]))
    dtest = xgb.DMatrix(test_X)
    model = load_pickle_model(model_path)
    y_pred = model.predict(dtest)
    return y_pred


def get_structural_fp_xgboost_pred(model_path):
    test_X = np.array(list(data_test["structural_fp"]))
    dtest = xgb.DMatrix(test_X)
    model = load_pickle_model(model_path)
    y_pred = model.predict(dtest)
    return y_pred


def get_esm1b_ts_drfp_xgboost_pred(model_path):
    drfp = np.array(list(data_test["DRFP"]))
    esm1b_ts = np.array(list(data_test["ESM1b_ts"]))
    test_X = np.concatenate([drfp, esm1b_ts], axis=1)
    model = load_pickle_model(model_path)
    y_pred = model.predict(xgb.DMatrix(test_X))
    return y_pred


def get_esm1b_ts_diff_fp_xgboost_pred(model_path):
    drfp = np.array(list(data_test["difference_fp"]))
    esm1b_ts = np.array(list(data_test["ESM1b_ts"]))
    test_X = np.concatenate([drfp, esm1b_ts], axis=1)
    model = load_pickle_model(model_path)
    y_pred = model.predict(xgb.DMatrix(test_X))
    return y_pred


def evaluate_models_weight(weights, model_preds, true_values):
    weighted_avg = np.average(model_preds, weights=weights, axis=0)
    r2 = r2_score(true_values, weighted_avg)
    return -r2  # We want to maximize R2, so minimize -R2


def calculate_weighted_mean(model_preds, true_values):
    num_models = model_preds.shape[0]
    # Initial weights (for illustration, you can adjust this as needed)
    initial_weights = np.ones(num_models) / num_models
    print(initial_weights)
    print(initial_weights.shape)
    # Constraint: the sum of weights must be 1
    constraints = {"type": "eq", "fun": lambda w: np.sum(w) - 1.0}
    # Bounds for weights (between 0 and 1)
    bounds = [(0, 1)] * num_models
    print("bounds shape:", bounds)
    result = minimize(
        evaluate_models_weight,
        initial_weights,
        args=(model_preds, true_values),
        method="SLSQP",
        bounds=bounds,
        constraints=constraints,
    )
    best_weights = result.x
    weighted_avg_pred = np.average(model_preds, weights=best_weights, axis=0)
    return weighted_avg_pred


def get_final_results(y_test, y_pred):
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    Pearson, _ = stats.pearsonr(np.reshape(y_test, -1), y_pred)
    output = {
        "Mean Squared Error": mse,
        "R-squared": r2,
        "Pearson coefficient": Pearson,
    }
    print(f"Ensemble model output:\n{output}")
    return output


def ensemble_xgboost_models():
    esm1b_pred = get_esm1b_xgboost_pred(
        get_model_path("xgboost_esm1b_r2_39_mse_81.h5"))

    esm1b_ts_pred = get_esm1b_ts_xgboost_pred(
        get_model_path("xgboost_esm1b_ts_r2_41_mse_77.h5")
    )
    diff_fp_pred = get_diff_xgboost_pred(
        get_model_path("xgboost_diff_r2_35_mse_86.h5")
        )

    structural_fp_pred = get_structural_fp_xgboost_pred(
        get_model_path("xgboost_structFp_r2_31_mse_91.h5")
    )

    drfp_fp_pred = get_drfp_xgboost_pred(
        get_model_path("xgboost_drfp_r2_35_mse_86.h5")
    )

    esm1b_ts_drfp_pred = get_esm1b_ts_drfp_xgboost_pred(
        get_model_path("xgboost_esm1bts_drfp_r2_43_mse_76_pc_65.h5")
    )

    esm1b_ts_diff_pred = get_esm1b_ts_diff_fp_xgboost_pred(
        get_model_path("xgboost_esm1b_ts_diff_r2_41_mse_77.h5")
    )

    models_pred = np.array(
        [
            esm1b_pred,
            esm1b_ts_pred,
            diff_fp_pred,
            structural_fp_pred,
            drfp_fp_pred,
            esm1b_ts_drfp_pred,
            esm1b_ts_diff_pred,
        ]
    )

    weighted_avg_pred = calculate_weighted_mean(models_pred, test_Y)
    get_final_results(test_Y, weighted_avg_pred)


if __name__ == "__main__":
    ensemble_xgboost_models()

[0.14285714 0.14285714 0.14285714 0.14285714 0.14285714 0.14285714
 0.14285714]
(7,)
bounds shape: [(0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1)]
Ensemble model output:
{'Mean Squared Error': 0.700865090031929, 'R-squared': 0.4757674632702046, 'Pearson coefficient': 0.6974826296893911}


## Convolutional Neural Network Models

In [ ]:
def get_esm1bts_norm_diff_cnn_predict(model_path):
    # create input matrices:
    test_X = np.array(list(data_test["difference_fp"]))
    test_X = np.concatenate(
        [test_X, np.array(list(data_test["ESM1b_ts_norm"]))], axis=1
    )

    model = load_model(model_path)
    y_pred = model.predict(test_X).reshape(-1)

    std_y, mean_y = np.std(test_Y), np.mean(test_Y)
    y_pred_final = (y_pred * std_y) + mean_y
    get_final_results(test_Y, y_pred_final)
    return y_pred_final


def get_esm1bts_norm_drfp_cnn_predict(model_path):
    # create input matricess
    test_X = np.array(list(data_test["DRFP"]))
    test_X = np.concatenate(
        [test_X, np.array(list(data_test["ESM1b_ts_norm"]))], axis=1
    )

    loaded_model = load_model(model_path)
    y_pred = loaded_model.predict(test_X).reshape(-1)
    std_y, mean_y = np.std(test_Y), np.mean(test_Y)
    y_pred_final = (y_pred * std_y) + mean_y

    return y_pred_final


def get_esm1b_norm_diff_cnn_predict(model_path):
    # Best hyperparameters: {'filters_1': 6, 'filters_2': 6, 'filters_3': 16, 'kernel_size_1': 13, 'kernel_size_2': 5, 'kernel_size_3': 13, 'dense_units_1': 256, 'dense_units_2': 8, 'dropout_rate': 0.5, 'optimizer': 'nadam', 'batch_size': 24}
    test_X = np.array(list(data_test["difference_fp"]))
    test_X = np.concatenate([test_X, np.array(list(data_test["ESM1b_norm"]))], axis=1)

    model = load_model(model_path)
    y_pred = model.predict(test_X).reshape(-1)
    std_y, mean_y = np.std(test_Y), np.mean(test_Y)
    y_pred_final = (y_pred * std_y) + mean_y
    return y_pred_final


def get_esm1b_norm_drfp_cnn_predict(model_path):
    test_X = np.array(list(data_test["DRFP"]))
    test_X = np.concatenate([test_X, np.array(list(data_test["ESM1b_norm"]))], axis=1)

    model = load_model(model_path)
    y_pred = model.predict(test_X).reshape(-1)
    std_y, mean_y = np.std(test_Y), np.mean(test_Y)
    y_pred_final = (y_pred * std_y) + mean_y
    return y_pred_final


def get_esm1bts_norm_cnn_predict(model_path):
    # create input matrices:
    test_X = np.array(list(data_test["ESM1b_ts_norm"]))

    model = load_model(model_path)
    y_pred = model.predict(test_X).reshape(-1)

    std_y, mean_y = np.std(test_Y), np.mean(test_Y)
    y_pred_final = (y_pred * std_y) + mean_y
    return y_pred_final


def get_esm1bts_norm_struct_fp_pred(model_path):
    structual_fp = np.array(list(data_test["structural_fp"]))
    esm1bts = np.array(list(data_test["ESM1b_ts_norm"]))
    test_X = np.concatenate([structual_fp, esm1bts], axis=1)

    model = load_model(model_path)
    y_pred = model.predict(test_X).reshape(-1)

    std_y, mean_y = np.std(test_Y), np.mean(test_Y)
    y_pred_final = (y_pred * std_y) + mean_y
    return y_pred_final


def get_esm1b_norm_struct_fp_pred(model_path):
    structual_fp = np.array(list(data_test["structural_fp"]))
    esm1b = np.array(list(data_test["ESM1b_norm"]))
    test_X = np.concatenate([structual_fp, esm1b], axis=1)

    model = load_model(model_path)
    y_pred = model.predict(test_X).reshape(-1)

    std_y, mean_y = np.std(test_Y), np.mean(test_Y)
    y_pred_final = (y_pred * std_y) + mean_y
    return y_pred_final


def evaluate_models_weight(weights, model_preds, true_values):
    weighted_avg = np.average(model_preds, weights=weights, axis=0)
    r2 = r2_score(true_values, weighted_avg)
    return -r2  # We want to maximize R2, so minimize -R2


def calculate_weighted_mean(model_preds, true_values):
    num_models = model_preds.shape[0]
    # Initial weights (for illustration, you can adjust this as needed)
    initial_weights = np.ones(num_models) / num_models
    # Constraint: the sum of weights must be 1
    constraints = {"type": "eq", "fun": lambda w: np.sum(w) - 1.0}
    # Bounds for weights (between 0 and 1)
    bounds = [(0, 1)] * num_models

    print()

    result = minimize(
        evaluate_models_weight,
        initial_weights,
        args=(model_preds, true_values),
        method="SLSQP",
        bounds=bounds,
        constraints=constraints,
    )
    best_weights = result.x
    weighted_avg_pred = np.average(model_preds, weights=best_weights, axis=0)
    return weighted_avg_pred


def get_final_results(y_test, y_pred):
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    Pearson, _ = stats.pearsonr(np.reshape(y_test, -1), y_pred)
    output = {
        "Mean Squared Error": mse,
        "R-squared": r2,
        "Pearson coefficient": Pearson,
    }
    print(f"Ensemble model output:\n{output}")
    return output


def ensemble_cnn_models():
    ESM1b_ts_norm_Diff_cnn_pred = get_esm1bts_norm_diff_cnn_predict(
        "../../models/best_models/cnn/cnn_esm1bts_norm_diff_r2_48.h5"
    )

    ESM1b_ts_norm_drfp_cnn_pred = get_esm1bts_norm_drfp_cnn_predict(
        "../../models/best_models/cnn/cnn_esm1bts_norm_drfp_r2_478.h5"
    )

    ESM1b_norm_diff_cnn_pred = get_esm1b_norm_diff_cnn_predict(
        "../../models/best_models/cnn/cnn_esm1b_diff_r2_47.h5"
    )

    ESM1b_norm_drfp_cnn_pred = get_esm1b_norm_drfp_cnn_predict(
        "../../models/best_models/cnn/cnn_esm1b_drfp_r2_48.h5"
    )

    ESM1b_ts_norm_struct_fp_cnn_pred = get_esm1bts_norm_struct_fp_pred(
        "../../models/best_models/cnn/cnn_esm1b_struct_r2_457.h5"
    )
    ESM1b_norm_struct_fp_cnn_pred = get_esm1b_norm_struct_fp_pred(
        "../../models/best_models/cnn/cnn_esm1b_struct_r2_46.h5"
    )


    cnn_preds = np.array(
        [
            ESM1b_ts_norm_Diff_cnn_pred,
            ESM1b_norm_diff_cnn_pred,
            ESM1b_ts_norm_drfp_cnn_pred,
            ESM1b_norm_drfp_cnn_pred,
            ESM1b_ts_norm_struct_fp_cnn_pred,
            ESM1b_norm_struct_fp_cnn_pred,
        ],
    )
    weighted_avg_pred = calculate_weighted_mean(np.array(cnn_preds), test_Y)
    get_final_results(test_Y, weighted_avg_pred)


if __name__ == "__main__":
    ensemble_cnn_models()
